# 📌 **Autenticación con Tokens JWT (SimpleJWT para manejar sesiones seguras)**  

El objetivo de este tema es que aprendas a manejar **autenticación segura** en Django usando **JSON Web Tokens (JWT)** con la librería **SimpleJWT**.  

---

## 🔹 **1. ¿Qué es JWT y por qué usarlo?**  

### 🔍 **JSON Web Token (JWT)**  
JWT es un estándar de autenticación basado en **tokens**, lo que permite a los usuarios iniciar sesión y recibir un token que luego envían en cada petición para identificarse.  

### 📌 **Ventajas de JWT frente a sesiones tradicionales**  
✔️ **Sin almacenamiento en servidor:** No es necesario guardar sesiones en la base de datos.  
✔️ **Más seguro:** Se puede verificar sin necesidad de hacer una consulta a la base de datos en cada solicitud.  
✔️ **Compatible con frontend y apps móviles:** Ideal para **React, Angular, Vue, Flutter, etc.**  

---

## 🔹 **2. Instalando SimpleJWT**  

Ejecutamos el siguiente comando:  

```bash
pip install djangorestframework-simplejwt
```

Luego, en `settings.py`, agregamos la configuración para usar SimpleJWT:  

```python
from datetime import timedelta

REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': (
        'rest_framework_simplejwt.authentication.JWTAuthentication',
    ),
}

SIMPLE_JWT = {
    'ACCESS_TOKEN_LIFETIME': timedelta(minutes=30),  # Expira en 30 minutos
    'REFRESH_TOKEN_LIFETIME': timedelta(days=1),  # Expira en 1 día
    'ROTATE_REFRESH_TOKENS': True,
    'BLACKLIST_AFTER_ROTATION': True,
}
```

📌 **¿Qué hacen estas opciones?**  
✔️ `ACCESS_TOKEN_LIFETIME`: Define cuánto dura un token antes de expirar.  
✔️ `REFRESH_TOKEN_LIFETIME`: Define cuánto dura el token de refresco para pedir un nuevo token.  
✔️ `ROTATE_REFRESH_TOKENS`: Si es `True`, cada vez que se usa un **refresh token**, se genera uno nuevo.  
✔️ `BLACKLIST_AFTER_ROTATION`: Evita que un refresh token viejo sea reutilizado.  

---

## 🔹 **3. Creando las rutas para autenticación JWT**  

En `urls.py`, agregamos las rutas para obtener y refrescar tokens:  

```python
from django.urls import path
from rest_framework_simplejwt.views import (
    TokenObtainPairView,
    TokenRefreshView,
)

urlpatterns = [
    path('api/token/', TokenObtainPairView.as_view(), name='token_obtain_pair'),  # Obtener token
    path('api/token/refresh/', TokenRefreshView.as_view(), name='token_refresh'),  # Refrescar token
]
```

📌 **¿Cómo funcionan estas rutas?**  

1️⃣ `POST /api/token/` → Devuelve un token de acceso y un refresh token.  
2️⃣ `POST /api/token/refresh/` → Usa el refresh token para obtener un nuevo token de acceso.  

---

## 🔹 **4. Probando la autenticación JWT**  

### 📍 **Obteniendo un token**  

Enviamos una solicitud `POST` a `http://127.0.0.1:8000/api/token/` con el usuario y contraseña:  

```json
{
    "username": "admin",
    "password": "123456"
}
```

📌 **Respuesta esperada:**  

```json
{
    "refresh": "eyJhbGciOiJIUzI1NiIs...",
    "access": "eyJhbGciOiJIUzI1NiIs..."
}
```

📌 **¿Cómo usarlo?**  
Para acceder a las rutas protegidas, enviamos el token en los headers:  

```
Authorization: Bearer <TOKEN>
```

---

## 🔹 **5. Protegiendo vistas con autenticación JWT**  

Si queremos que solo usuarios autenticados accedan a ciertas vistas, usamos `IsAuthenticated`.  

📍 **Ejemplo en `views.py`:**  

```python
from rest_framework.permissions import IsAuthenticated
from rest_framework.views import APIView
from rest_framework.response import Response

class VistaProtegida(APIView):
    permission_classes = [IsAuthenticated]

    def get(self, request):
        return Response({"mensaje": "¡Estás autenticado!"})
```

📍 **Registramos la ruta en `urls.py`:**  

```python
from django.urls import path
from .views import VistaProtegida

urlpatterns = [
    path('api/protegido/', VistaProtegida.as_view(), name='vista_protegida'),
]
```

📌 **Si intentamos acceder sin token (http://127.0.0.1:8000/api/protegido/), la API responderá con:**  

```json
{
    "detail": "Las credenciales de autenticación no se proporcionaron."
}
```

Pero si enviamos el token en los headers, podremos acceder.  

---

## 🔹 **6. Logout con Blacklist (Opcional pero recomendado)**  

Por defecto, Django REST Framework no **invalida** un token cuando un usuario cierra sesión. Para eso, usamos un sistema de **Blacklist**.  

📍 **Paso 1: Agregamos `rest_framework_simplejwt.token_blacklist` a `INSTALLED_APPS` en `settings.py`**  

```python
INSTALLED_APPS = [
    'rest_framework',
    'rest_framework_simplejwt.token_blacklist',
]
```

📍 **Paso 2: Ejecutamos la migración**  

```bash
python manage.py migrate
```

📍 **Paso 3: Creamos una vista para hacer logout (`views.py`)**  

```python
from rest_framework_simplejwt.tokens import RefreshToken
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework.permissions import IsAuthenticated

class LogoutView(APIView):
    permission_classes = [IsAuthenticated]

    def post(self, request):
        try:
            refresh_token = request.data["refresh"]
            token = RefreshToken(refresh_token)
            token.blacklist()
            return Response({"mensaje": "Sesión cerrada correctamente."})
        except Exception as e:
            return Response({"error": "Token inválido."}, status=400)
```

📍 **Paso 4: Agregamos la ruta en `urls.py`**  

```python
from .views import LogoutView

urlpatterns += [
    path('api/logout/', LogoutView.as_view(), name='logout'),
]
```

📌 **Flujo del logout:**  
✔️ El usuario envía su `refresh_token` a `POST /api/logout/`.  
✔️ Django lo agrega a la lista negra, impidiendo su reutilización.  

---

## ✅ **Resumen del flujo de trabajo con SimpleJWT**  

1️⃣ Instalamos **SimpleJWT** y lo configuramos en `settings.py`.  
2️⃣ Creamos las rutas para **obtener y refrescar tokens**.  
3️⃣ Protegemos vistas con `IsAuthenticated`.  
4️⃣ Implementamos **logout con blacklist** para invalidar tokens.  